In [ ]:
import csv
import time
import requests

# URL ของ endpoint ที่มีข้อมูล JSON
url = 'http://192.168.4.1/data'  # เปลี่ยนเป็น URL จริง

interval = 0.125  # อ่านข้อมูลทุก 0.125 วินาที
filename = "data_collected_idle-f25.csv"

# ใช้ requests.Session() เพื่อให้เปิด TCP Connection ค้างไว้ ลด Overhead
session = requests.Session()

# สร้างไฟล์ CSV และเขียน header
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ax", "ay", "az", "gx", "gy", "gz"])  # เขียน header

    print("Starting data collection... (Press Ctrl + C to stop)")

    try:
        start_time = time.perf_counter()
        next_read_time = start_time  # เวลาที่ต้องอ่านข้อมูลครั้งแรก

        while True:
            current_time = time.perf_counter()

            if current_time >= next_read_time:
                try:
                    request_start = time.perf_counter()
                    response = session.get(url, timeout=1, stream=False)  # ดึง JSON แบบเร็วขึ้น
                    request_end = time.perf_counter()

                    if response.status_code == 200:
                        data = response.json()  # แปลงข้อมูลเป็น JSON
                        print(f"Raw data: {data} (Request time: {request_end - request_start:.4f}s)")

                        # ตรวจสอบว่าได้ข้อมูลครบทั้ง 6 ค่า
                        if all(key in data for key in ["ax", "ay", "az", "gx", "gy", "gz"]):
                            row = [data["ax"], data["ay"], data["az"], data["gx"], data["gy"], data["gz"]]
                            writer.writerow(row)  # เขียนข้อมูลลงในไฟล์
                            print(f"Data written: {row}")

                        else:
                            print("Incomplete data received")

                    else:
                        print(f"Failed to fetch data: {response.status_code}")

                    next_read_time += interval  # กำหนดเวลาอ่านข้อมูลครั้งถัดไป

                except requests.RequestException as e:
                    print(f"Request error: {e}")

            # ปรับเวลาหน่วงให้แม่นยำขึ้น
            time.sleep(max(0, next_read_time - time.perf_counter()))

    except KeyboardInterrupt:
        print("\nInterrupted by user.")

    finally:
        session.close()
        print("Data collection complete. File saved as", filename)


Starting data collection...


: 